In [3]:
from osgeo import gdal
import rasterio as rio
import rasterio.mask
import geopandas as gpd
import subprocess
import numpy as np 
import matplotlib.pyplot as plt

In [4]:
def align_rasters(base_raster, raster_to_align, output_file):
    # Open base raster to get its projection, resolution, and bounds
    base_ds = gdal.Open(base_raster)
    projection = base_ds.GetProjection()
    geotransform = base_ds.GetGeoTransform()
    resolution = (geotransform[1], geotransform[5])
    bounds = (geotransform[0], 
              geotransform[3], 
              geotransform[0] + geotransform[1] * base_ds.RasterXSize,
              geotransform[3] + geotransform[5] * base_ds.RasterYSize)
    
    
    # Build the gdalwarp command
    gdalwarp_cmd = [
        'gdalwarp',
        '-te', f"{bounds[0]}", f"{bounds[1]}", f"{bounds[2]}", f"{bounds[3]}",
        '-tr', f"{resolution[0]}", f"{resolution[1]}",
        '-s_srs', "EPSG:2278",
        '-t_srs', projection,
        '-r', 'bilinear',
        '-overwrite',
        raster_to_align,
        output_file
    ]

    # Run gdalwarp
    subprocess.run(gdalwarp_cmd)
    print(f"Raster aligned and saved to {output_file}")
    


In [7]:
base_dir = r"C:\Users\franc\OneDrive - University Of Houston\AAA_Flume\SpringBayou\Terrain\\"
shapes_dir  = r"C:\Users\franc\OneDrive - University Of Houston\AAA_Flume\SpringBayou\Terrain\Streams_Buffered.gpkg"

terrain = f"{base_dir}Terrain_FilledGrid.tif"
# xs1     = f"{base_dir}CrossSections_100.tif"
xs2     = f"{base_dir}XS.tif"
# xs1_p   = f"{base_dir}CrossSections_100_proj.tif"
xs2_p   = f"{base_dir}XS_proj.tif"
    
if True:
    # align_rasters(terrain, xs1, xs1_p)
    align_rasters(terrain, xs2, xs2_p)

Raster aligned and saved to C:\Users\franc\OneDrive - University Of Houston\AAA_Flume\SpringBayou\Terrain\\XS_proj.tif


In [8]:
def burnStreams(terrain_fn, xs_fn, mask=None, convert_ft=False):
    if isinstance(terrain_fn, str):
        terrain_fn = rio.open(terrain_fn)
        terrain = np.array(terrain_fn.read(1))
    else:
        terrain = terrain_fn
        
    if convert_ft:
        terrain = terrain * 3.28084
    
    xs_ds = rio.open(xs_fn)
    
    if mask is not None:
        shapes = gpd.read_file(shapes_dir)
        shapes_dis = shapes.dissolve()
        xs, xs_t = rio.mask.mask(xs_ds, shapes_dis[['geometry']].values.flatten())
        
        
    else:
        xs = np.array(xs.read(1))
        
    
    xs[xs == xs_ds.nodata] = 9999
    xs = xs[:, ::-1, :]
    
    terrain = np.where(xs < terrain, xs, terrain)
    return terrain, xs

In [11]:
# terrain_new, xs1 = burnStreams(terrain, xs1_p, mask=shapes_dir, convert_ft=True)
terrain_new2, xs2 = burnStreams(terrain, xs2_p, mask=shapes_dir, convert_ft=True)

In [12]:
savedir = f"{base_dir}Terrain_burned.tif"
with rio.open(terrain) as src:
    with rio.open(savedir, 'w', **src.profile) as dst:
        dst.write(terrain_new2[0, :, :], 1)